In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from pathlib import Path
import torch
import torch.nn as nn
from timm import create_model  # For EfficientNet-B5
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# Step 1: Load Data
csv_path = '/kaggle/input/hackathondatasetsecond/train_data.csv'
image_folder = '/kaggle/input/hackathondatasetsecond/train/train'
data = pd.read_csv(csv_path)

In [ ]:
# Map city names to numeric labels
label_mapping = {"Istanbul": 0, "Ankara": 1, "Izmir": 2}
data['city_label'] = data['city'].map(label_mapping)

In [ ]:
# Stratified Split into Train and Validation Sets
train_data, val_data = train_test_split(
    data, test_size=0.2, stratify=data['city_label'], random_state=42
)


In [ ]:
# Add file paths
train_data['file_path'] = train_data['filename'].apply(lambda x: str(Path(image_folder) / x))
val_data['file_path'] = val_data['filename'].apply(lambda x: str(Path(image_folder) / x))

In [ ]:
# Step 2: Define Augmentations
# Training Augmentations
train_transforms = transforms.Compose([
    transforms.Resize((380, 380)),  # Resize for EfficientNet-B5
    # transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
    # transforms.RandomRotation(degrees=10),  # Random rotations
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Validation and Test Augmentations
val_transforms = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Step 3: Custom Dataset Class
class CityDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['file_path']
        label = self.dataframe.iloc[idx]['city_label']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label
# Step 4: Create Datasets and DataLoaders
train_dataset = CityDataset(train_data, transform=train_transforms)
val_dataset = CityDataset(val_data, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)

In [ ]:
# Step 5: Define the Model
class CityClassifier(nn.Module):
    def __init__(self):
        super(CityClassifier, self).__init__()
        self.base_model = create_model('efficientnet_b4', pretrained=True)
        num_features = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(num_features, 3)  # Output layer for 3 classes
        )

    def forward(self, x):
        return self.base_model(x)


In [ ]:
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CityClassifier().to(device)


In [ ]:
# Step 6: Define Loss Function, Optimizer, and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)

def train_one_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    running_loss, true_labels, pred_labels = 0.0, [], []
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())
    return running_loss / len(data_loader), f1_score(true_labels, pred_labels, average='macro')

def validate_one_epoch(model, data_loader, criterion, device):
    model.eval()
    running_loss, true_labels, pred_labels = 0.0, [], []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            true_labels.extend(labels.cpu().numpy())
            pred_labels.extend(preds.cpu().numpy())
    return running_loss / len(data_loader), f1_score(true_labels, pred_labels, average='macro')

# Step 8: Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=10):
    train_losses, val_losses, train_f1s, val_f1s = [], [], [], []
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        train_loss, train_f1 = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_f1 = validate_one_epoch(model, val_loader, criterion, device)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_f1s.append(train_f1)
        val_f1s.append(val_f1)
        scheduler.step()
        print(f"Train Loss: {train_loss:.4f}, Train F1: {train_f1:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}")
    return train_losses, val_losses, train_f1s, val_f1s

In [ ]:
# Train the Model
train_losses, val_losses, train_f1s, val_f1s = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=10
)

In [ ]:
# Step 9: Test Inference and Submission
test_csv_path = '/kaggle/input/hackathondatasetsecond/test.csv'
test_image_folder = '/kaggle/input/hackathondatasetsecond/test/test'

test_data = pd.read_csv(test_csv_path)
test_data['file_path'] = test_data['filename'].apply(lambda x: str(Path(test_image_folder) / x))

test_transforms = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class TestDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['file_path']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.dataframe.iloc[idx]['filename']

test_dataset = TestDataset(test_data, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

model.eval()
predictions = []
with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        predictions.extend(zip(filenames, preds))

submission = pd.DataFrame(predictions, columns=['filename', 'city'])
submission['city'] = submission['city'].map({0: "Istanbul", 1: "Ankara", 2: "Izmir"})
submission.to_csv('submission.csv', index=False)
print("Submission file created!")